In [84]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("spark://cm1:7077").appName("LifeGame00001").getOrCreate()


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
# import numpy as np
import time

# Funções do código original
def ind2d(i, j, tam):
    return i * (tam + 2) + j

def calculaVida(x, tabulIn):
    i = x[0]
    j = x[1]
    tam = x[2]
    if (i == 0 or j == 0 or i == tam + 1 or j == tam + 1):
        return 0
    
    vizviv = tabulIn[ind2d(i - 1, j - 1, tam)] + tabulIn[ind2d(i - 1, j, tam)] + \
             tabulIn[ind2d(i - 1, j + 1, tam)] + tabulIn[ind2d(i, j - 1, tam)] + \
             tabulIn[ind2d(i, j + 1, tam)] + tabulIn[ind2d(i + 1, j - 1, tam)] + \
             tabulIn[ind2d(i + 1, j, tam)] + tabulIn[ind2d(i + 1, j + 1, tam)]
    if tabulIn[ind2d(i, j, tam)] and vizviv < 2:
        return 0
    elif tabulIn[ind2d(i, j, tam)] and vizviv > 3:
        return 0
    elif not tabulIn[ind2d(i, j, tam)] and vizviv == 3:
        return 1
    else:
        return tabulIn[ind2d(i, j, tam)]
    

def UmaVida(tam, tabulIn):
    tabul_out = []
    for i in range(0, tam + 2):
        for j in range(0, tam + 2):
            tabul_out.append([i, j, tam])
            
    rdd_tabul_out = spark.sparkContext.parallelize(tabul_out).map(lambda x: calculaVida(x, tabulIn))    
        
    return rdd_tabul_out.collect()

def InitTabul(tam):
    tabulIn = [0] * (tam + 2) * (tam + 2)
    
    tabulIn[ind2d(1, 2, tam)] = 1;
    tabulIn[ind2d(2, 3, tam)] = 1;
    tabulIn[ind2d(3, 1, tam)] = 1;
    tabulIn[ind2d(3, 2, tam)] = 1;
    tabulIn[ind2d(3, 3, tam)] = 1;
    return tabulIn

def Correto(tam, tabul):
    cnt = sum(tabul)
    return (cnt == 5 and tabul[ind2d(tam - 2, tam - 1, tam)] and
            tabul[ind2d(tam - 1, tam, tam)] and tabul[ind2d(tam, tam - 2, tam)] and
            tabul[ind2d(tam, tam - 1, tam)] and tabul[ind2d(tam, tam, tam)])

def process_board(tam):
    t0 = time.time()
    tabulIn = InitTabul(tam)
    
    for i in range(2 * (tam - 3)):
        tabulOut = UmaVida(tam, tabulIn)
        tabulIn = UmaVida(tam, tabulOut)
    t1 = time.time()
    return tabulIn, t1 - t0

# Configuração do ambiente Spark


# Número de tamanhos do tabuleiro a serem processados
POWMIN = 3
POWMAX = 10

# Para todos os tamanhos do tabuleiro, processa em paralelo
results = []
for pow in range(POWMIN, POWMAX + 1):
    tam = 1 << pow
    tabul, time_elapsed = process_board(tam)
    if Correto(tam, tabul):
        print(f" tam={tam} **Ok, RESULTADO CORRETO**")
        print(f" time={time_elapsed}")
    else:
        print(f" tam={tam} **Nok, RESULTADO ERRADO**")
        print(f" time={time_elapsed}")
    

# Encerra o Spark
spark.stop()


 tam=8 **Ok, RESULTADO CORRETO**
 time=8.283813238143921
 tam=16 **Ok, RESULTADO CORRETO**
 time=8.375973463058472
 tam=32 **Ok, RESULTADO CORRETO**
 time=18.700819969177246
 tam=64 **Ok, RESULTADO CORRETO**
 time=40.72215127944946
 tam=128 **Ok, RESULTADO CORRETO**
 time=86.22166776657104
 tam=256 **Ok, RESULTADO CORRETO**
 time=234.15412831306458
 tam=512 **Ok, RESULTADO CORRETO**
 time=954.4769561290741
